In [1]:
!pip install pandas
!pip install tslearn
!pip install fbprophet
!pip install pmdarima
import pandas as pd
import os
import pandas as pd 
import numpy as np 
from sklearn.metrics import mean_squared_error as mse
import FORECAST_1001 
import FORECAST_1031 
import transform
# import FORECAST_1036
import FORECAST_1034
import traintest
import FORECAST_1035
# import FORECAST_1004

  Using cached fbprophet-0.7.1.tar.gz (64 kB)
  Using cached cmdstanpy-0.9.5-py3-none-any.whl (37 kB)
  Created wheel for fbprophet: filename=fbprophet-0.7.1-py3-none-any.whl size=723195 sha256=0962635af0d1cbd1c85d8711e0b4b1f08247d20ddb2d1a72b90ba2649144a329
  Stored in directory: /Users/parthshah/Library/Caches/pip/wheels/d0/d2/ae/c579b7fd160999d35908f3cb8ebcad7ef64ecaca7b78e4c3c8
Successfully built fbprophet
  Attempting uninstall: cmdstanpy
    Found existing installation: cmdstanpy 0.9.68
    Uninstalling cmdstanpy-0.9.68:
      Successfully uninstalled cmdstanpy-0.9.68
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
prophet 1.0.1 requires cmdstanpy==0.9.68, but you have cmdstanpy 0.9.5 which is incompatible.


In [2]:
df_train = pd.read_excel("Sales_Data.xlsx")

In [3]:
df_train['DATE'].min()

Timestamp('2014-04-01 00:00:00')

In [20]:
df_train[["DATE","ID"]].groupby("ID").count()

,DATE
ID,
East,2133
North,2133
Overall,2133
South 1,2133
South 2,2133
West,2133


In [4]:
import numpy as np  
import itertools

def expandgrid(*itrs):
   product = list(itertools.product(*itrs))
   return {'Var{}'.format(i+1):[x[i] for x in product] for i in range(len(itrs))}



In [5]:
import FORECAST_1001 

dataExtraction__An error occurred while extracting and merging the 2 dataframes.


UnboundLocalError: local variable 'salesDF' referenced before assignment

In [6]:
import config
category = "J"
config.DICT_JEW_PATH[category]

'Sales_Data.xlsx'

In [2]:
import numpy as np  
import itertools

def expandgrid(*itrs):
   product = list(itertools.product(*itrs))
   return {'Var{}'.format(i+1):[x[i] for x in product] for i in range(len(itrs))}

#level parameter must be case sensitive to the column name in the csv/excel file 
def extractdata(salesdata_filepath,flags_filepath,level='ID' , salesDF = None,freq="D"):
    level=str(level)
    #importing flags and sales data 
    #flags=pd.read_csv(str(flags_filepath))
    flags = selectFileFromPath(str(flags_filepath))
    
    if not salesdata_filepath == "":
        #salesdata = pd.read_csv(str(salesdata_filepath))
        salesdata = selectFileFromPath(str(salesdata_filepath))
    else:
        salesdata = salesDF
    

    #Converting DATE in string format to datetime format 
    flags['DATE'] = pd.to_datetime(flags['DATE'],format='%d-%m-%Y')
    salesdata['DATE'] = pd.to_datetime(salesdata['DATE'],format='%d-%m-%Y')

    #Extracting min and max DATE from dataset and creating index with all dates in between 
    mindate=salesdata['DATE'].min()
    maxdate=salesdata['DATE'].max()
    date_rng = pd.date_range(start=mindate, end=maxdate, freq=freq)

    #Extracting all unique regions/stores/cat/PB
    unique=set(salesdata.loc[:,level])

    #Creating df with all combinations of level and DATE
    completedata=expandgrid(unique,date_rng)
    completedf=pd.DataFrame(completedata)

    #Appropriately renaming columnns of new df
    completedf=completedf.rename(columns={'Var1':'ID','Var2':'DATE'})
    
    #Merging new df with flags and sales dataset 
    #completedf=pd.merge(flags,completedf,on='DATE')
    completedf=pd.merge(completedf,salesdata,how='left',left_on=["ID",'DATE'],right_on=[level,'DATE'])
    
    #Setting DATE as index of df 
    completedf = completedf.set_index('DATE')
    #completedf=completedf.drop(level,axis=1)
    #print(completedf.head())
    completedf.columns=['ID',"SALES"]
    flags = flags.set_index('DATE')
    
    #Replacing na with 0 
    sales=completedf.fillna(0)
    flags=flags.fillna(0)
    return sales, flags



In [3]:
def dataExtraction(category , level , salesFromPath = True , flagsFromPath = True):
    
    try:

        if salesFromPath == True and flagsFromPath == True:
            salesDF , flagsDF = extractdata(config.DICT_JEW_PATH[category] , config.DICT_FLAG_PATH[category] , level)
        elif salesFromPath == False:
            if category == "E":
                salesDF = selectData(dictSelectEyewareData[level])
            elif category == "W":
                salesDF = selectData(dictSelectWatchData[level])
            elif category == "J":
                salesDF = selectData(dictSelectJewelData[level])

            dictColNames = {"OVR" : "Overall" , "REG" : "Regional" , "STO" : "Store" , "SKU" : "SKU" , "CAT" : "Category" , "CAP" : "CAT PB" }                          

            salesDF , flagsDF = extractdata("" , config.DICT_FLAG_PATH[category] , dictColNames[level] , salesDF)
        
        return salesDF , flagsDF
    except:
        print("dataExtraction__An error occurred while extracting and merging the 2 dataframes.")
        raise



In [4]:
def traintestsplit(data,train=0.7):
    try:
        train_size=int(len(data)*train)
        train, test = data.iloc[:train_size,], data.iloc[train_size:,]
        return train,test 
    
    except:
        print("traintestsplit__An error occured while splitting data into train and test")



In [5]:
level="ID",
frequency="M",
n=1,
salesFromPath=True,
flagsFromPath=True,
transformation="",
confidencelevel=0.95

In [7]:
def MAPE(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true.sum() - y_pred.sum()) / y_true.sum())) * 100

sales, flags = FORECAST_1001.dataExtraction(category = "J", level = "ID",salesFromPath = True,
                                                 flagsFromPath = True)

unique=set(sales["ID"])
algotable = pd.DataFrame()
forecast = pd.Series()
############################## DATA PRE-PROCESSING ########################
#Looping over every individual item in col "ID"
for unit in unique:
    #Subsetting sales and flags data for each unique item in "ID" column of
    #salesdata df for training and forecasting 
    print("Starting forecast for CATPB: ",unit)
    salesdata = pd.Series(sales.loc[sales["ID"]==unit,"SALES"])
    startdate = salesdata.index[0]
    #startdate = salesdata[np.min(salesdata):].index[0]
    salesdata = salesdata[startdate:]
    flagsdata = flags[startdate:]  

    #Grouping data to a weekly level for training/forecasting
    if frequency=="W":
        salesdata = salesdata.resample('W').sum()
        flagsdata = flagsdata.drop('WEEKEND_FLAG',axis=1,errors='ignore')
        flagsdata = flagsdata.resample('W').mean()

    #Grouping data to a monthly level for training/forecasting
    if frequency=="M":
        salesdata = salesdata.resample('M').sum()
        flagsdata = flagsdata.drop(["Start of Month Flag","MONTHEND_FLAG",
                                    "End of Month Flag"],axis=1,errors='ignore')
        flagsdata = flagsdata.resample('M').mean()   
    n = 1    
    flagsdf = flagsdata.iloc[len(salesdata):len(salesdata)+n]   
    flagsdata = flagsdata.iloc[:len(salesdata)]        

    ######### TRAIN-TEST SPLIT #########
    trainsales, testsales = traintestsplit(salesdata)
    trainflags, testflags = traintestsplit(flagsdata)

    ############################# MODEL TRAINING ############################# 
    #creating dataframe for storing mse for models 
    errortable = pd.DataFrame()
    

    ########## ARIMA ##########
    #Transforming data to stationarize it 
    transformeddata = stationarize(trainsales,conflev=0.95)
    #Passing scaled data to the arimax model 
    scaledforecast = FORECAST_1034.arimax(transformeddata[0],len(testsales))
    #Inverse scaling the scaled forecasts 
    arimaforecasts = transform.inverse(scaledforecast,transformeddata)
    error = mse(arimaforecasts ,testsales)
    error2 = MAPE(testsales,arimaforecasts)
    #Appending mse and MAPE to error table 
    errortable = errortable.append({'Algorithm' : 'ARIMA', 'mse' : error, 
                                    "MAPE":error2},ignore_index=True)            
    del scaledforecast, arimaforecasts


selectFileFromPath__An error occurred while fetching file from path.
selectFileFromPath__An error occurred while fetching file from path.
dataExtraction__An error occurred while extracting and merging the 2 dataframes.


TypeError: 'NoneType' object is not subscriptable

In [23]:
errortable

,Algorithm,MAPE,mse
0,ARIMA,3.319490e+01,1.495159e+12
1,ARIMAX,2.161639e+14,2.877609e+38
2,HOLT-WINTERS-AANd,NaN,1.000000e+15


In [8]:
def MAPE(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true.sum() - y_pred.sum()) / y_true.sum())) * 100


#Passing scaled data to the arimax model 
scaledforecast = FORECAST_1034.arimax(transformeddata[0],len(testsales))
#Inverse scaling the scaled forecasts 
arimaforecasts = transform.inverse(scaledforecast,transformeddata)
error = mse(arimaforecasts ,testsales)
error2 = MAPE(testsales,arimaforecasts)
#Appending mse and MAPE to error table 
errortable = errortable.append({'Algorithm' : 'ARIMA', 'mse' : error, 
                                "MAPE":error2},ignore_index=True)            
del scaledforecast, arimaforecasts



NameError: name 'transformeddata' is not defined

In [20]:
trainflags1 = trainflags.iloc[-len(transformeddata[0]):,:]
transformeddata[0].index = trainflags1.index
scaledforecast = FORECAST_1034.arimax(transformeddata[0],len(testsales),
                             trainFlags=trainflags1,forecastFlags=testflags)
arimaxforecasts = transform.inverse(scaledforecast,transformeddata)
error = mse(arimaxforecasts ,testsales)
error2 = MAPE(testsales,arimaxforecasts)
errortable = errortable.append({'Algorithm' : 'ARIMAX', 'mse' : error,
                                "MAPE":error2},ignore_index=True)
del transformeddata, scaledforecast, arimaxforecasts, trainflags1



In [22]:
######### HOLT-WINTERS #########
category = "J"
holtMSE,algo = FORECAST_1031.HOLTWINTER(trainsales,testsales,category)
if algo == "AANd":
    #AANd = Additive trend , Additive seasonality , No Damp
    errortable = errortable.append({'Algorithm' : 'HOLT-WINTERS-AANd', 
                                    'mse' : holtMSE},ignore_index=True)                                     

elif algo == "AMNd":    
    #AANd = Additive trend , Multiplicative seasonality , No Damp
    errortable = errortable.append({'Algorithm' : 'HOLT-WINTERS-AMNd', 
                                    'mse' : holtMSE},ignore_index=True)                                 
elif algo == "AAD":
    #AAD = Additive trend , Additive seasonality , With Damp
    errortable = errortable.append({'Algorithm' : 'HOLT-WINTERS-AAD', 
                                    'mse' : holtMSE},ignore_index=True)                                          
elif algo == "AMD":
    #AMD = Additive trend , Multiplicative seasonality , No Damp
    errortable = errortable.append({'Algorithm' : 'HOLT-WINTERS-AMD', 
                                    'mse' : holtMSE},ignore_index=True)                                    



In [41]:
frequency

('M',)

In [39]:
#Holt winter algo implementation for forecasting
def seasonalityPeriod(category, data_freq = "D"):
    m = config.DICT_SEASONALITY[category][data_freq]
    return m


def HOLTWINTER(train, test ,category , data_freq = "D" , only_error = True, algoIndex = 0):
    #Get seasonality period
    m = seasonalityPeriod(category,data_freq)


In [42]:
HOLTWINTER(trainsales,testsales,category)

In [44]:
errortable

,Algorithm,MAPE,mse
0,ARIMA,3.222314e+00,2.117674e+12
1,ARIMAX,6.630659e+14,7.182185e+39
2,HOLT-WINTERS-AANd,NaN,1.000000e+15


In [27]:
flagsdata.iloc[len(salesdata):len(salesdata)+n] 

TypeError: unsupported operand type(s) for +: 'int' and 'tuple'

In [34]:
n = 1
len(salesdata)+ n

2134

In [15]:
#Applying various transformations to data to stationarize it
def stationarize(data,conflev=0.95):
    #1st order difference
    diffdata=transform.difference(data)
    output=transform.adfuller(diffdata)
    pval = output[1]
    if pval>=(1-conflev):
        #2nd order difference
        diff2data=difference(diffdata)
        output=adfuller(diff2data)
        pval = output[1]
        if pval<(1-conflev):
            #print("Data transformation: 2nd order differencing")
            transformation = "dd"
            initial_values= [data.iloc[-1],diffdata.iloc[-1]]
            return diff2data,transformation, initial_values
    else:
        #print("Data transformation: 1st order differencing")
        transformation = "d"
        initial_value= data.iloc[-1]
        return diffdata,transformation, initial_value

    #Log transformation
    if not np.any(data==0):
        logdata=np.log(data)
        output=adfuller(logdata)
        pval = output[1]
        #log+differencing
        if pval>=(1-conflev):
            difflogdata=difference(logdata)
            output=adfuller(difflogdata)
            pval = output[1]
            if pval<(1-conflev):
                #print("Data transformation: Log + 1st order differencing")
                transformation = "ld"
                intial_value = logdata.iloc[-1]
                return difflogdata, transformation, intial_value
        else:
            #print("Data transformation: Log")
            transformation = "l"
            return logdata,transformation
    else: 
        data = -data
        expdata= np.exp(data)
        output=adfuller(expdata)
        pval = output[1]
        if pval<(1-conflev):
            #print("Data transformation: Exponential")
            transformation = "e"
            return expdata, transformation
        
    boxcoxdata  = stats.boxcox(data)
    output=adfuller(boxcoxdata[0])
    pval = output[1]
    if pval<(1-conflev):
        #print("Data transformation: Box-Cox")
        transformation = "bc"
        return boxcoxdata[0], transformation, boxcoxdata[1]



In [16]:
transformeddata = stationarize(trainsales,conflev=0.95)

In [17]:
transformeddata[0]

0       -153207.29
1        -66406.99
2        344137.81
3        412407.88
4         73347.95
           ...    
1487     616561.56
1488     744913.32
1489    -361161.65
1490    2292146.36
1491   -4405460.21
Length: 1492, dtype: float64

In [23]:
FORECAST_1034.arimax(transformeddata[0],len(testsales))

1492    1.259057e+06
1493    4.910867e+04
1494   -1.044443e+05
1495    5.842879e+05
1496   -5.880066e+05
            ...     
2127    9.530599e+02
2128    9.530599e+02
2129    9.530599e+02
2130    9.530599e+02
2131    9.530599e+02
Length: 640, dtype: float64

In [22]:
transformeddata[0]

0        177067.20
1       -151249.70
2        451862.95
3       1408482.92
4      -1211466.87
           ...    
1487    -361090.50
1488    1472891.00
1489   -1423051.45
1490    2519655.00
1491   -3516002.58
Length: 1492, dtype: float64

In [24]:
transformeddata = transform.transform(trainsales,confidencelevel,
                                          transformation)

In [25]:
transformeddata[0]

TypeError: 'NoneType' object is not subscriptable